<a href="https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/create_animation_full_mlo_record_keelingcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create an animation of a graphic appearing on the Keeling Curve website
### **Animation: CO<sub>2</sub> full record at Mauna Loa, Hawaii**

#### This notebook creates a customizable mp4 animation of a graphic appearing on the front page of [keelingcurve.ucsd.edu](keelingcurve.ucsd.edu).

See an overview of the Keeling Curve notebooks and how to use them at [notebooks overview](https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/overview_of_notebooks_keelingcurve.ipynb)

Direct questions to [webmaster-co2o2-sio@ucsd.edu](mailto:webmaster-co2o2-sio@ucsd.edu)

----

### [Go to creating the animation section](#function-to-create-and-save-a-bare-animation)

-----

### Sample of the animation produced by this notebook

In [34]:
from base64 import b64encode
from IPython.display import HTML, display
import requests
import pathlib
from ipywidgets import widgets
import functools
import math

def show_sample_animation(anim_out, btn):

    anim_out.clear_output()

    video_url = 'https://github.com/sio-co2o2/keelingcurve_notebooks/blob/main/images/sample_animations/mlo_full_record.mp4?raw=1'

    r = requests.get(video_url, stream = True)

    download_dir = pathlib.Path('./downloads')
    download_dir.mkdir(exist_ok=True)

    video_name = f'./downloads/mlo_full_record.mp4'

    with open(video_name,"wb") as f:
        for chunk in r.iter_content(chunk_size=1024):
            # writing one chunk at a time 
            if chunk:
                f.write(chunk)
            
    the_animation = open(video_name,'rb').read()

    data_url = "data:video/mp4;base64," + b64encode(the_animation).decode()

    html_str = """
    <video width=800 controls>
            <source src="%s" type="video/mp4">
    </video>
    """ % data_url

    with anim_out:
        display(HTML(html_str))


anim_out = widgets.Output()

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Show sample animation',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)  

button.on_click(functools.partial(show_sample_animation, anim_out))
display(button)

display(anim_out)

Button(button_style='primary', description='Show sample animation', layout=Layout(height='auto', width='auto')…

Output()

## **Notebook Code**

### Import python packages

In [35]:
import functools
from time import perf_counter
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import pathlib
import matplotlib.image as mpimg
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator)
from matplotlib import ticker
from matplotlib import animation
from ipywidgets import widgets
from IPython.display import HTML, display
from base64 import b64encode
import os

%matplotlib inline

In [36]:
# This package is used to convert a vector svg into a png

try:
  from cairosvg import svg2png
except:
  ! pip install cairosvg
  from cairosvg import svg2png

In [37]:
# This import is to enable Google Colab to save files ane then download them

# This import does not exist unless the notebook is run in Google Colab
# Put in a try except block if user wants to use notebook off of Google Colab

try:
    from google.colab import files
except:
    pass


### Set directories and file names

In [38]:
# Read in data from github repository
# Get mlo record to last daily value
data_file = "https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/preliminary_data/mlo/mlo_full_record_now_span.csv"

logo_file = "https://github.com/sio-co2o2/keelingcurve_notebooks/raw/main/images/logos/ucsd_sio_logo.svg"

plot_basename = "mlo_full_record"

animation_dir = pathlib.Path("./animations")
animation_dir.mkdir(exist_ok=True)

animation_file = f'{plot_basename}.mp4'


## **Load in data and prepare it for plotting**

### Load in MLO record

In [39]:
df = pd.read_csv(data_file, skipinitialspace=True, comment='"')
df.head()


,date,co2,date_seas_adj,co2_seas_adj,date_fit,co2_fit,date_seas_adj_fit,co2_seas_adj_fit,date_filled,co2_filled,date_seas_adj_filled,co2_seas_adj_filled
0,1958.202740,315.71,1958.202740,314.43,1958.202740,316.20,1958.202740,314.91,1958.202740,315.71,1958.202740,314.43
1,1958.287671,317.45,1958.287671,315.16,1958.287671,317.30,1958.287671,314.99,1958.287671,317.45,1958.287671,315.16
2,1958.369863,317.51,1958.369863,314.69,1958.369863,317.88,1958.369863,315.07,1958.369863,317.51,1958.369863,314.69
3,1958.536986,315.87,1958.536986,315.20,1958.454795,317.26,1958.454795,315.15,1958.454795,317.26,1958.454795,315.15
4,1958.621918,314.93,1958.621918,316.21,1958.536986,315.86,1958.536986,315.22,1958.536986,315.87,1958.536986,315.20


### Break into 2 dataframes. One for CO2 and one for the CO2 fit

In [40]:
df_co2 = df[["date", "co2"]].copy()
df_co2_fit = df[["date_fit", "co2_fit"]].copy()


### Remove rows with fill values -99.99 or NaN

In [41]:
df_co2 = df_co2[df_co2["co2"] != -99.99]
df_co2_fit = df_co2_fit[df_co2_fit["co2_fit"] != -99.99]

df_co2 = df_co2.dropna()
df_co2_fit = df_co2_fit.dropna()

df_co2.tail()


,date,co2
798,2023.208219,419.96
799,2023.209589,420.48
800,2023.212329,420.33
801,2023.226027,420.57
802,2023.228767,420.73


### Convert to numpy arrays for plotting

In [42]:
date_co2 = df_co2["date"].to_numpy()
co2 = df_co2["co2"].to_numpy()

date_co2_fit = df_co2_fit["date_fit"].to_numpy()
co2_fit = df_co2_fit["co2_fit"].to_numpy()


## Extract out Maunakea (MKO) data

Needed to differentiate MKO data when plotting

### Get start and end decimal dates of MKO data (12/1/2022 to 3/8/2023)

In [43]:
mauna_kea_start_date = datetime(2022, 12, 1)

year = mauna_kea_start_date.year
boy = datetime(year, 1, 1)
eoy = datetime(year + 1, 1, 1)
mko_decimal_start_date = year + ((mauna_kea_start_date - boy).total_seconds() / ((eoy - boy).total_seconds()))

mauna_kea_end_date = datetime(2023, 3, 8)

year = mauna_kea_end_date.year
boy = datetime(year, 1, 1)
eoy = datetime(year + 1, 1, 1)
mko_decimal_end_date = year + ((mauna_kea_end_date - boy).total_seconds() / ((eoy - boy).total_seconds()))


### Filter data to get MKO data

In [44]:
df_co2_mko = df_co2[(df_co2['date'] >= mko_decimal_start_date) & (df_co2['date'] <= mko_decimal_end_date)]

df_co2_fit_mko = df_co2_fit[(df_co2_fit['date_fit'] >= mko_decimal_start_date) & (df_co2_fit['date_fit'] <= mko_decimal_end_date)]

### Convert to numpy arrays for plotting

In [45]:
date_co2_mko = df_co2_mko['date'].to_numpy()
co2_mko = df_co2_mko['co2'].to_numpy()

date_co2_fit_mko = df_co2_fit_mko['date_fit'].to_numpy()
co2_fit_mko = df_co2_fit_mko['co2_fit'].to_numpy()

## **Define accessory functions**

## Functions to convert dates into various formats
Used for plot limits and labeling

In [46]:
# Function to convert datetime to a float
def dt2t(adatetime):
    """
    Convert adatetime into a float. The integer part of the float should
    represent the year.
    Order should be preserved. If adate<bdate, then d2t(adate)<d2t(bdate)
    time distances should be preserved: If bdate-adate=ddate-cdate then
    dt2t(bdate)-dt2t(adate) = dt2t(ddate)-dt2t(cdate)
    """
    year = adatetime.year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    return year + ((adatetime - boy).total_seconds() / ((eoy - boy).total_seconds()))


In [47]:
def get_todays_date_variations():

    now = datetime.now()
    todays_decimal = dt2t(now)

    today = date.today()

    todays_day = today.day
    todays_month = today.strftime("%B")
    todays_year = today.year
    todays_date_moyr = today.strftime("%B %Y")
    todays_date_modyyr = f"{todays_month} {todays_day}, {todays_year}"

    return todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year


In [48]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def t2dt(atime):
    """
    Convert atime (a float) to DT.datetime
    This is the inverse of dt2t.
    assert dt2t(t2dt(atime)) == atime
    """
    year = int(atime)
    remainder = atime - year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    seconds = remainder * (eoy - boy).total_seconds()
    return boy + timedelta(seconds=seconds)


In [49]:
def get_data_end_date(date_data):
    last_date = np.max(date_data)
    last_date_datetime = t2dt(last_date)

    # Convert date format to month_name day, year
    day = last_date_datetime.day
    month = last_date_datetime.strftime("%B")
    year = last_date_datetime.year
    date_modyyr = f"{month} {day}, {year}"
    return date_modyyr


## **Define functions that can modify the animated graphic**

### Function to set fonts and linewidth properties

In [50]:
def set_matplotlib_properties():

    # Set properties for matplotlib

    # Reset rcparams in case modified defaults in another notebook during same session
    plt.rcParams.update(plt.rcParamsDefault)

    plt.rcParams.update({"axes.linewidth": 1.5})

    plt.rcParams.update(
        {
            "text.usetex": False,
            "font.family": "sans-serif",
            "font.weight": "normal",
            "font.sans-serif": [
                "Arial",
                "Tahoma",
                "Helvetica",
                "FreeSans",
                "NimbusSans",
                "LiberationSans",
                "DejaVu Sans",
            ],
            "mathtext.default": "regular",
            "mathtext.fontset": "dejavusans",
        }
    )

    # http://phyletica.org/matplotlib-fonts/
    # This tells matplotlib to use Type 42 (a.k.a. TrueType) fonts
    # for PostScript and PDF files. This allows you to avoid Type 3 fonts.
    # Turning on usetex also works
    # Needed on Ubuntu for pdf fonts
    plt.rcParams.update({"pdf.fonttype": 42, "ps.fonttype": 42})


### Function to set axes properties of the graphic

In [51]:
def set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel):

    # ---------------------------------
    # Plot properties for website plots
    # ---------------------------------

    # Allow room at top for the 2 titles
    fig.subplots_adjust(top=0.85)

    ax.tick_params(which="both", bottom=True, top=True, left=True, right=True)

    ax.tick_params(axis="x", labelsize=12)
    ax.tick_params(axis="y", labelsize=16)

    ax.tick_params(axis="x", which="major", direction="in", length=8, width=1)
    ax.tick_params(axis="y", which="major", direction="in", length=8, width=1)

    tick_spacing = 5
    ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

    tick_spacing = 5
    ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

    labels = ax.get_xticklabels() + ax.get_yticklabels()
    for label in labels:
        label.set_fontweight("bold")

    # Display every other y major tick label
    for label in ax.yaxis.get_ticklabels()[::2]:
        label.set_visible(False)

    tick_length = 5
    ax.tick_params(axis="x", which="minor", direction="in", length=tick_length)

    tick_length = 4
    ax.tick_params(axis="y", which="minor", direction="in", length=tick_length)

    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))

    ax.set_xlabel(xlabel, fontweight="bold", fontsize=18, labelpad=5)
    ax.set_ylabel(ylabel, fontweight="bold", fontsize=18, labelpad=5)

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)


### Function to place the titles

In [52]:
def add_plot_title(ax, title1, title2):

    ax.annotate(title1, xy=(0, 1.15), xycoords='axes fraction', fontsize=14,
                horizontalalignment='left', verticalalignment='top', fontweight="normal")
    ax.annotate(title2, xy=(0, 1.07), xycoords='axes fraction', fontsize=18,
                horizontalalignment='left', verticalalignment='top', fontweight = 'bold')

### Function to add an inset label

In [53]:
def add_inset_label(ax, label_start, today):
    label_text = f"{label_start} ending {today}"
    ax.annotate(
        label_text,
        xy=(0.03, 0.9),
        xycoords="axes fraction",
        fontsize=12,
        horizontalalignment="left",
        verticalalignment="top",
        fontweight="normal",
    )


### Function to add the SIO logo

In [54]:
def add_sio_logo(fig, logo_file, xpos, ypos):

    # Convert the logo svg file to a png file with the
    # given scale and dpi
    logo_png = "logo.png"
    svg2png(url=logo_file, write_to=logo_png, scale=10, dpi=300)

    logo = mpimg.imread(logo_png)

    fig.add_axes([xpos, ypos, 0.2, 0.2], anchor="SE", zorder=1)

    plt.imshow(logo)
    plt.axis("off")

    # Delete the logo png version
    os.remove(logo_png)


### Function to create a Context Manager

This allows a message to display in a cell while a function is running.
Usefull for indicating a function is running.

In [55]:
class ShowProgressContextManagerWidgets:

    def __init__(self, out, command=''):
        self.out = out
        self.command = command
        
    def __enter__(self):
        css_style = "<style>span.start { margin:0; padding: 0; color: blue; }</style>"
        html_str = f'<span class="start">Running {self.command}...</span>'

        self.out.clear_output()
        
        with self.out:
            display(HTML(css_style))
            display(HTML(html_str))

        self.time = perf_counter()

        return self

        
    def __exit__(self, exc_type, exc_value, exc_tb):
        
        css_style = "<style>span.end { margin: 0; padding: 0; color: green; }</style>"
        html_str = '<span class="end">Finished</span>'

        self.out.clear_output()

        with self.out:
            display(HTML(css_style)) 
            display(HTML(html_str))

            self.time = perf_counter() - self.time
            self.readout = f'Time to create animation: {self.time:.3f} seconds'
            print(self.readout)



# **Steps to create the animation**

### Comment out any function in the overall function ['create_animation'](#function-to-create-the-animation) to remove an element from the final animation

Comment out a line by adding a # symbol in front of a statement


## **Set properties to use for matplotlib (the plotting engine)**

In [56]:
set_matplotlib_properties()


## **Set plot limits and labeling**

### Set titles and axes labels

In [57]:
xlabel = ""
ylabel = "$\mathregular{CO}\\bf{_2}$" + " Concentration (ppm)"

title1 = ""
title2 = "Carbon dioxide concentration at Mauna Loa Observatory"


### Set yaxis min and max limits

In [58]:
ymin = 310
ymax = 430


### Set xaxis min and max limits

In [59]:
xmin = min(date_co2)

# Get current date as decimal year
now = datetime.now()
xmax = dt2t(now)


<a name="function-to-create-and-save-a-bare-animation"></a>
## **Function to create and save a bare animation**

Create a bare animation without titles, legends, and sio logo

In [60]:
def create_animation_bare(msg_out, anim_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_co2_fit, co2_fit, fps, dpi, btn):

    # create a figure container
    # and set aspect ration of the animation (width, height)
    fig = plt.figure(figsize=(10,6))

    ax =fig.gca()

    fig_width, fig_height = fig.get_size_inches()

    # set width in inches
    width_in = 10
    height_in = (fig_height / fig_width) * width_in

    fig.set_size_inches(width_in, height_in)

    fig.subplots_adjust(left=0.11, bottom=0.1, right=0.96, top=0.56, wspace=None, hspace=None)

    # ---------------------------------------------------
    # Context manager 
    # 
    # Used to show messages while creating the animiation
    # ---------------------------------------------------

    with ShowProgressContextManagerWidgets(msg_out, 'Create Animation') as progress:

        # -------------------------------------------
        # Create the plot but don't display it
        #
        # The variable 'line_of_plot' is used 
        # to display pieces of the plot at each frame
        # -------------------------------------------

        mlo_line, = ax.plot([], [], color='black', lw=2)
        mko_line, = ax.plot([], [], color='blue', lw=2)

        lines = []
        lines.append(mlo_line)
        lines.append(mko_line)


        # ------------------------------------------------
        # Get todays date in various formats for labeling
        # ------------------------------------------------
        (
            todays_date_moyr,
            todays_date_modyyr,
            todays_decimal,
            todays_year,
        ) = get_todays_date_variations()

        # ---------------
        # Add plot titles
        # ---------------
        #add_plot_title(ax, title1, title2)

        # --------------
        # Add inset text
        # --------------
        #add_inset_label(ax, "Full record", todays_date_modyyr)

        # --------------------
        # Add SIO logo to plot
        # --------------------

        # set position of the logo
        xpos = 0.715
        ypos = 0.155

        #add_sio_logo(fig, logo_file, xpos, ypos)

        # ----------------------------------------
        # Set plot properties

        # Set after the other figure properties to
        # update the default properties
        # -----------------------------------------
        set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel)


        # ====================
        # Create the animation
        # ====================

        # -----------------------------
        # function to update each frame
        # -----------------------------
        blue = '#1e47b0'

        def init():
            for line in lines:
                line.set_data([],[])
            return lines

        number_of_frames = len(date_co2_fit)

        # For testing
        # mko_decimal_start_date = 1961.0
        # number_of_frames = 70

        frame_interval = 1
        
        total_frame_time = date_co2_fit[-1] - date_co2_fit[0]
        one_frame_time = math.ceil(total_frame_time/number_of_frames + 1)

        for frame_number in range(0, number_of_frames, frame_interval):
            if date_co2_fit[frame_number] >= mko_decimal_start_date:
                mko_frame_number = frame_number
                break

        def update(frame_number):

            if frame_number >= mko_frame_number:

                lines[1].set_data(date_co2_fit[mko_frame_number - one_frame_time:frame_number], co2_fit[mko_frame_number - one_frame_time :frame_number
                ])

            else:
                lines[0].set_data(date_co2_fit[:frame_number], co2_fit[:frame_number])

            return lines 

        anim = animation.FuncAnimation(fig, func=update, frames = number_of_frames, interval=frame_interval, init_func=init, blit=False, save_count=number_of_frames+100)

        # When testing, use a samaller set of frames
        # anim = animation.FuncAnimation(fig, func=update, frames = number_of_frames, interval=frame_interval, init_func=init, blit=False)

        # Save the animation
        anim.save(f'{animation_dir}/{animation_file}', fps=fps, dpi=dpi)

        # display the animation in the output cell
        the_animation = open(f'{animation_dir}/{animation_file}','rb').read()

        with anim_out:

            data_url = "data:video/mp4;base64," + b64encode(the_animation).decode()
            
            html_str = """
            <video width=800 controls>
                    <source src="%s" type="video/mp4">
            </video>
            """ % data_url
            
            display(HTML(html_str))

<a name='button-to-create-and-save-a-bare-animation'></a>

## **Button to create and save a bare animation**

**The animation takes about 2 minutes on a local computer and about 4 minutes using Google Colab to create at fps=60 and dpi=72**

In [61]:
msg_out = widgets.Output()
anim_out = widgets.Output()

# frames per secod
fps = 60

# dpi of animation
dpi = 72

button = widgets.Button(description='Create animation', button_style='primary')
button.on_click(functools.partial(create_animation_bare, msg_out, anim_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_co2_fit, co2_fit, fps, dpi))
display(button)

msg_out.clear_output()
anim_out.clear_output()

display(msg_out)
display(anim_out)


Button(button_style='primary', description='Create animation', style=ButtonStyle())

Output()

Output()

## **Button to download the bare animation file**

In [62]:
def download_files(msg_out, btn):

    msg_out.clear_output()

    try:
        files.download(animation_file)
    except:
        with msg_out:
            print("Need to create the animation first.")


msg_out = widgets.Output()

button = widgets.Button(description="Download Animation File", button_style="primary")

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Download Animation File',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)   

button.on_click(functools.partial(download_files, msg_out))

display(button)
display(msg_out)

Button(button_style='primary', description='Download Animation File', layout=Layout(height='auto', width='auto…

Output()

<a name="function-to-create-and-save-the-animation"></a>
## **Function to create and save an animation with more features**

Include title, legend, and logo

In [63]:
def create_animation(msg_out, anim_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_co2_fit, co2_fit, fps, dpi, btn):

    # create a figure container
    # and set aspect ration of the animation (width, height)
    fig = plt.figure(figsize=(10,6))

    ax =fig.gca()

    fig_width, fig_height = fig.get_size_inches()

    # set width in inches
    width_in = 10
    height_in = (fig_height / fig_width) * width_in

    fig.set_size_inches(width_in, height_in)

    fig.subplots_adjust(left=0.11, bottom=0.1, right=0.96, top=0.56, wspace=None, hspace=None)

    # ---------------------------------------------------
    # Context manager 
    # 
    # Used to show messages while creating the animiation
    # ---------------------------------------------------

    with ShowProgressContextManagerWidgets(msg_out, 'Create Animation') as progress:

        # -------------------------------------------
        # Create the plot but don't display it
        #
        # The variable 'line_of_plot' is used 
        # to display pieces of the plot at each frame
        # -------------------------------------------

        mlo_line, = ax.plot([], [], color='black', lw=2)
        mko_line, = ax.plot([], [], color='blue', lw=2)

        # line starts with mlo data, then mko data, and finally mlo data
        lines = []
        lines.append(mlo_line)
        lines.append(mko_line)
        lines.append(mlo_line)


        # ------------------------------------------------
        # Get todays date in various formats for labeling
        # ------------------------------------------------
        (
            todays_date_moyr,
            todays_date_modyyr,
            todays_decimal,
            todays_year,
        ) = get_todays_date_variations()

        # ---------------
        # Add plot titles
        # ---------------
        add_plot_title(ax, title1, title2)

        # --------------
        # Add inset text
        # --------------
        add_inset_label(ax, "Full record", todays_date_modyyr)

        # --------------------
        # Add SIO logo to plot
        # --------------------

        # set position of the logo
        xpos = 0.715
        ypos = 0.155

        add_sio_logo(fig, logo_file, xpos, ypos)

        # ----------------------------------------
        # Set plot properties

        # Set after the other figure properties to
        # update the default properties
        # -----------------------------------------
        set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel)


        # ====================
        # Create the animation
        # ====================

        # -----------------------------
        # function to update each frame
        # -----------------------------
        blue = '#1e47b0'

        def init():
            for line in lines:
                line.set_data([],[])
            return lines

        number_of_frames = len(date_co2_fit)

        # When testing, use a samaller set of frames and change mko range to fit
        # mko_decimal_start_date = 1961.0
        # mko_decimal_end_date = 1962.0
        # number_of_frames = 70

        frame_interval = 1

        total_frame_time = date_co2_fit[-1] - date_co2_fit[0]
        one_frame_time = math.ceil(total_frame_time/number_of_frames + 1)

        mko_frames = []

        for frame_number in range(0, number_of_frames, frame_interval):

            if (date_co2_fit[frame_number] >= mko_decimal_start_date) and (date_co2_fit[frame_number] <= mko_decimal_end_date):
                mko_frames.append(frame_number)
        
        mko_start_frame_number = mko_frames[0]
        mko_end_frame_number = mko_frames[-1]

        def update(frame_number):

            if frame_number < mko_start_frame_number:
                lines[0].set_data(date_co2_fit[:frame_number], co2_fit[:frame_number])

            elif frame_number >= mko_start_frame_number and frame_number <= mko_end_frame_number:

                lines[1].set_data(date_co2_fit[mko_start_frame_number - one_frame_time:frame_number], co2_fit[mko_start_frame_number - one_frame_time :frame_number
                ])
            
            else:
                lines[2].set_data(date_co2_fit[:frame_number], co2_fit[:frame_number])

            return lines 

        anim = animation.FuncAnimation(fig, func=update, frames = number_of_frames, interval=frame_interval, init_func=init, blit=False, save_count=number_of_frames+100)

        # testing
        # Uncomment the anim line and comment the above anim line for testing
        # Use a smaller number of frames set above and can also increase the
        # frame interval
        # anim = animation.FuncAnimation(fig, func=update, frames = number_of_frames, interval=frame_interval, init_func=init, blit=False)

        # Save the animation
        anim.save(f'{animation_dir}/{animation_file}', fps=fps, dpi=dpi)

        # display the animation in the output cell
        the_animation = open(f'{animation_dir}/{animation_file}','rb').read()

        with anim_out:

            data_url = "data:video/mp4;base64," + b64encode(the_animation).decode()
            
            html_str = """
            <video width=800 controls>
                    <source src="%s" type="video/mp4">
            </video>
            """ % data_url
            
            display(HTML(html_str))

        

<a name='button-to-create-and-save-the-animation'></a>

## **Button to create and save the animation**

**The animation takes about 5 minutes on a local computer and about 10 minutes using Google Colab to create at fps=60 and dpi=72**

In [64]:
msg_out = widgets.Output()
anim_out = widgets.Output()

# frames per secod
fps = 60

# dpi of animation
dpi = 72

button = widgets.Button(description='Create animation', button_style='primary')
button.on_click(functools.partial(create_animation, msg_out, anim_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_co2_fit, co2_fit, fps, dpi))
display(button)

msg_out.clear_output()
anim_out.clear_output()

display(msg_out)
display(anim_out)

# If the animation is not created, there is most likely an error in the animation functions used
# to create the plot. Turn on console to diagnose errors.


Button(button_style='primary', description='Create animation', style=ButtonStyle())

Output()

Output()

## **Button to download the animation file**

In [65]:
def download_files(msg_out, btn):

    msg_out.clear_output()

    try:
        files.download(animation_file)
    except:
        with msg_out:
            print("Need to create the animation first.")


msg_out = widgets.Output()

button = widgets.Button(description="Download Animation File", button_style="primary")

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Download Animation File',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)   

button.on_click(functools.partial(download_files, msg_out))

display(button)
display(msg_out)

Button(button_style='primary', description='Download Animation File', layout=Layout(height='auto', width='auto…

Output()